In [3]:
import gym
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt

In [4]:
env = gym.make("CartPole-v1")
num_states = env.observation_space.shape[0]
num_actions = env.action_space.n
num_episodes = 500
gamma = 0.99
learning_rate = 0.001


In [5]:
class ActorCritic(tf.keras.Model):
    def __init__(self, num_actions):
        super().__init__()
        self.common = layers.Dense(32, activation="relu")
        self.actor = layers.Dense(num_actions, activation="softmax")
        self.critic = layers.Dense(1)

    def call(self, inputs):
        x = self.common(inputs)
        return self.actor(x), self.critic(x)

In [6]:
class QActorCriticAgent:
    def __init__(self, num_states, num_actions):
        self.num_states = num_states
        self.num_actions = num_actions
        self.model = ActorCritic(num_actions)
        self.optimizer = tf.keras.optimizers.Adam(learning_rate)

    def act(self, state):
        state = np.expand_dims(state, axis=0)
        action_probs, _ = self.model.predict(state)
        action = np.random.choice(self.num_actions, p=np.squeeze(action_probs))
        return action

    def update(self, state, action, reward, next_state, done):
        state = np.expand_dims(state, axis=0)
        next_state = np.expand_dims(next_state, axis=0)

        _, critic_value = self.model.predict(state)
        _, next_critic_value = self.model.predict(next_state)

        target = reward + gamma * next_critic_value * (1 - int(done))
        td_error = target - critic_value

        with tf.GradientTape() as tape:
            action_probs, critic_value = self.model(state)
            _, next_critic_value = self.model(next_state)

            target = reward + gamma * next_critic_value * (1 - int(done))
            td_error = target - critic_value

            log_action_probs = tf.math.log(action_probs)
            log_prob = tf.reduce_sum(
                tf.one_hot(action, self.num_actions) * log_action_probs, axis=1
            )

            actor_loss = -tf.reduce_mean(log_prob * td_error)
            critic_loss = td_error ** 2
            total_loss = actor_loss + 0.5 * critic_loss

        grads = tape.gradient(total_loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.model.trainable_variables))


In [13]:
agent = QActorCriticAgent(num_states, num_actions)
episode_rewards = []

for episode in range(num_episodes):
    state = env.reset()
    episode_reward = 0

    while True:
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        agent.update(state, action, reward, next_state, done)

        state = next_state
        episode_reward += reward

        if done:
            break

    episode_rewards.append(episode_reward)
    print(f"Episode {episode + 1}: reward = {episode_reward}")


In [14]:
import matplotlib.pyplot as plt

plt.plot(episode_rewards)
plt.xlabel("Episode")
plt.ylabel("Total Reward")
plt.show()
plt.savefig('Total_Reward_QAC_Cartpole')

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Testing the agent for 20 Episodes (Agent choosing only greedy actions from the learnt policy)

In [15]:
test_episode_rewards = []

for episode in range(20):
    state = env.reset()
    episode_reward = 0

    while True:
        action_probs, _ = agent.model.predict(np.expand_dims(state, axis=0))
        action = np.argmax(action_probs)

        next_state, reward, done, _ = env.step(action)

        state = next_state
        episode_reward += reward

        if done:
            break

    test_episode_rewards.append(episode_reward)
    print(f"Test episode {episode + 1}: reward = {episode_reward}")

plt.plot(test_episode_rewards)
plt.xlabel("Episode")
plt.ylabel("Total Reward")
plt.show()
plt.savefig('Testing_Total_Reward_QAC_Cartpole')